# Welcome  

Notebook Author: Samuel Alter  
Notebook Subject: Capstone Project - Image Analysis

BrainStation Winter 2023: Data Science

This notebook is for running my images through deep learning networks. The images have been pre-processed in a separate notebook.

Using a pre-trained satellite image classifier from `Kaggle`, I will analyze the my satellite data using a `fire` and `nofire` set of image(s)

`Kaggle` sources, choose one or many:

1. [Satellite Image classification by aravindanr](https://www.kaggle.com/code/aravindanr22052001/satellite-image-classification)

1. [Intel Image Classification by Puneet Bansal](https://www.kaggle.com/datasets/puneet6060/intel-image-classification)

1. [Image Classification using CNN (94%+ Accuracy) by ask9](https://www.kaggle.com/code/arbazkhan971/image-classification-using-cnn-94-accuracy)

1. [Satellite Image Classification by Mahmoud Reda](https://www.kaggle.com/datasets/mahmoudreda55/satellite-image-classification)

1. [🛰️Satellite Image Classification | CNN by Sujith K Mandala](https://www.kaggle.com/code/sujithmandala/satellite-image-classification-cnn)

I will start with <mark>number 1</mark> because it has the highest accuracy.

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import cv2


In [9]:
# from kaggle 1

# import cv2
# import numpy as np
# import pandas as pd 
# import matplotlib.pyplot as plt 

# import tensorflow as tf 
# from keras.models import Sequential 
# from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPooling2D, GlobalAveragePooling2D
# from keras.preprocessing.image import ImageDataGenerator, load_img 

# from IPython import display 
# display.set_matplotlib_formats('svg')

### Tutorial for [visualizing tiff files](https://www.geeksforgeeks.org/visualizing-tiff-file-using-matplotlib-and-gdal-using-python/)

In [15]:
from osgeo import gdal
import matplotlib.pyplot as plt

fire=r'/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/ortho_2018_sm_fire.tif'
nofire=r'/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/ortho_2018_sm_nofire.tif'
  
dataset_fire = gdal.Open(fire)

In [17]:
# print the number of bands

print(dataset_fire.RasterCount)

4


In [18]:
# since there are 4 bands
# we store in 4 different variables

band1_red = dataset.GetRasterBand(1) # Red channel
band2_gre = dataset.GetRasterBand(2) # Green channel
band3_blu = dataset.GetRasterBand(3) # Blue channel

In [19]:
# read bands as numpy arrays

b1 = band1_red.ReadAsArray()
b2 = band2_gre.ReadAsArray()
b3 = band3_blu.ReadAsArray()

In [ ]:
# warning: this won't work because the files are too large
# # plot the arrays

# img = np.dstack((b1, b2, b3))
# f = plt.figure()
# plt.imshow(img)
# plt.savefig('fire.png')
# plt.show()

In [6]:
# https://gis.stackexchange.com/questions/14712/splitting-raster-into-smaller-chunks-using-gdal

fire=r'/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/ortho_2018_sm_fire.tif'
nofire=r'/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/ortho_2018_sm_nofire.tif'

from osgeo import gdal
from gdalconst import *

width = 512
height = 512
tilesize = 64

for i in range(0, width, tilesize):
    for j in range(0, height, tilesize):
        gdaltranString = f"gdal_translate -of {fire} -srcwin "+str(i)+", "+str(j)+", "+str(tilesize)+", " \
            +str(tilesize)+" utm.tif utm_"+str(i)+"_"+str(j)+".tif"
        os.system(gdaltranString)

ModuleNotFoundError: No module named 'gdalconst'